In [1]:
import numpy as np
import sys,math
sys.path.append('/home/lukas/projects/antibiotics/programs/')
import platereaderclass as prc
from plates_EstimateMSP import estimate_Tau_sMIC_linearFit_AsFuncOfB

import matplotlib
import matplotlib.pyplot as plt
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

%matplotlib inline
matplotlib.rcParams['figure.figsize'] = [12,8]
matplotlib.rc('font',family='Monospace')


In [2]:
datafile1 = '/home/lukas/projects/antibiotics/experiments/curated_datafiles/august2019_philip.xlsx'
designassignment = [0,0,0,0,0,0,0,0, 0,0,1,1,1,1,0,0]

In [3]:
data = prc.PlateReaderData(infiles = [datafile1],DesignAssignment = designassignment)
threshold = data.EstimateGrowthThreshold(dataID = None)

In [13]:
PlotExtension = 10 ** 1.5
FitPoints = 40
np.seterr(all='ignore')

@interact
def ManualFit(Experiment = data.titles, Transitions = ['standard','GPR'],ShowInocula = True,IdenticalRanges = True,Tau_Exponent = (-8.,-3.),sMIC_Exponent = (-3.,3.),xi = (0.0,2.0,.1)):
    idx = data.titles.index(Experiment)
    
    if   Transitions == 'standard': tr = data.compute_growth_nogrowth_transition(idx,threshold)
    elif Transitions == 'GPR':      tr = data.compute_growth_nogrowth_transition_GPR(idx,threshold)
    
    platedesign = data.get_design(dataID = idx)
    if ShowInocula: plt.scatter(platedesign[0],platedesign[1],c='#d3d7cf',s=10)
    
    plt.yscale('log')
    plt.xscale('log')
    if IdenticalRanges:
        xra = (1e-2,1e4)
        yra = (1e0,1e8)
    else:
        xra = (np.min(platedesign[0])/PlotExtension,np.max(platedesign[0])*PlotExtension)
        yra = (np.min(platedesign[1])/PlotExtension,np.max(platedesign[1])*PlotExtension)

    plt.xlim(xra)
    plt.ylim(yra)
    
    npoints = np.exp(np.linspace(np.log(yra[0]),np.log(yra[-1]),num = FitPoints))
    bpoints = 10 ** (sMIC_Exponent) * np.exp(np.power(10 ** (Tau_Exponent) *(npoints - 1),1./xi))
     
    plt.plot(bpoints,npoints,c='#f57900')
    plt.scatter(tr[:,0],tr[:,1],s=40,c='#3465a4')
    plt.legend(['TAU = {:.3e}, sMIC = {:.3e}'.format(10**Tau_Exponent,10**sMIC_Exponent)])
    

interactive(children=(Dropdown(description='Experiment', options=('G238S None 1', 'G238S None 2', 'G238S High …